# Classify Industries

## IDEEN


### HTML Klassifizierung


- Text zusammenfassen und dann klassifizieren? Dafür auch HTML-Tags verwenden?
- ~~HTML Struktur verwenden, um vorher **Boilerplate Content** von Main Content zu entfernen:~~
    - ~~Plain Text ist sehr noisy (viel unnötiges drin)~~
    - ICH: gemacht mit CLEAN HTML, aber ohne explizites Boilerplate Content Removal
- Bestimmten Wörtern/Tags höhere Gewichtungen geben
    - Anchor Text (= klickbarer Text in einem Hyperlink)
        - alleine zu wenig Inhalt (QI, S. 12)
        - umliegende Wörter interessant! (QI, S. 12)
        - auch für Nachbar-Seiten-Ansatz
    - Title, Headers (QI, S. 12)
        - auch für Nachbar-Seiten-Ansatz
    - Keywords für Branchen
        ```python3  
        Class_1_keywords = ['Office', 'School', 'phone', 'Technology', 'Electronics', 'Cell', 'Business', 'Education', 'Classroom']
        
        Class_2_keywords = ['Restaurant', 'Hospitality', 'Tub', 'Drain', 'Pool', 'Filtration', 'Floor', 'Restroom', 'Consumer', 'Care', 'Bags', 'Disposables']
        
        Class_3_keywords = ['Pull', 'Lifts', 'Pneumatic', 'Emergency', 'Finishing', 'Hydraulic', 'Lockout', 'Towers', 'Drywall', 'Tools', 'Packaging', 'Measure', 'Tag ']
        ```
- NER mit **Tags** als weitere Tokens
- Features von "Nachbarseiten" verwenden
    - Hilfreich, da mehr Infos als "nur" Startseite
    - Fragen: 
        - Was sind Nachbarseiten, wie definieren?
            - Webgraph Webseiten?
            - Weitere Seiten des Unternehmens?
        - Wie viele Nachbarseiten?
        - Wieviel von den Nachbarseiten verwenden?
            - Ganze Seite?
            - text, title, heading, Metadaten?
    
- *Weiteres*:
    - Flat classification oder Hierarchical classification?
        - Flat: parallele Klassen
        - Hierarchical: hierarchische Klassen, bauen aufeinander auf
    - Nur nach bestimmten Keywords filtern? (das geht jedoch mehr Richtung PLAIN-Textclassification)
    - "implicit links": Seiten, die beide bei Suche von **Suchmaschine** erschienen sind und auf die beide der User geklickt hat (QI, S. 12) &rarr; nicht wirklich realisierbar




## Tests

- Evaluation metric: **F1 Scores**
- TF-IDF Vectorizer
    - kein lowercase
    - stop words werden entfernt
    - keine max features
- Top $n$ classes = most frequent classes
- CLEAN HTML auch für Test Set (ansonsten unglaublich schlechte Accuracy und etwas sinnlos)


#### Label: `group_representatives`

| Experiment | SGD F1 (Precision) | LSVM F1 (Precision) |
| ---------- |:-----:| ----:|
| HTML (10000 features) | **0.5292** (0.5962) | **0.5493** (0.6371) |
| HTML (kept stop words) (10000 features) | **0.5268** (0.5845) | **0.5473** (0.6439) |
| HTML (10000 features) ((1, 3) ngrams) | **0.4035** (0.463) | **0.4188** (0.5345) |
| HTML (10000 features) ((2, 2) ngrams) | **0.2442** (0.2787) | **0.252** (0.3146) |
| ---------- |-----| ----|
| *ALL LANGS* HTML (kept stop words) (10000 features) | **0.5781** (0.6464) | **0.6406** (0.7024) |
| ---------- |-----| ----|
| Plain Text (kept stop words) (10000 features) (10000 rows) | **0.5841** (0.6301) | **0.5778** (0.6257) |
| Plain Text + Meta (kept stop words) (10000 features) (10000 rows) | **0.5832** (0.6197) | **0.5826** (0.6279) |

In [32]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# sklearn classification
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC

# sklearn general
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import (confusion_matrix, 
                             classification_report, 
                             f1_score, 
                             precision_score,
                             recall_score)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder


from stop_words import get_stop_words
import ujson as json


import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from app.utils import (clean_boilerplate, 
                       clean_website, 
                       detect_XML, 
                       extract_meta_informations,
                       remove_tags, 
                       tokenizing_html, 
                       trim_html)

## Paths

In [49]:
DATA_DIR_PATH = "../data/"
LANG = ""
ROWS = "_10000"

INDUSTRIES_PATH_CSV = DATA_DIR_PATH + "industries.csv"
TRAIN_PATH_CSV = DATA_DIR_PATH + "train" + LANG + ROWS + ".csv"
TEST_PATH_CSV = DATA_DIR_PATH + "test" + LANG + ROWS +".csv"

## Load train csv

In [50]:
%%time
train = pd.read_csv(TRAIN_PATH_CSV)
train = train.fillna("")

CPU times: user 2.5 s, sys: 383 ms, total: 2.88 s
Wall time: 2.87 s


In [51]:
train.head(1)

,url,industry,industry_label,group,group_representative,html,text,source,country,group_representative_label,meta
0,http://www.autarctech.de,144,Renewables & Environment,"gov, man, org",144,<html> <head> ...,Home\n\nMenü\n\n\nShop\nHome\nProdukte\nOur St...,xing,DE,Renewables & Environment,Effizenz bei der Stromspeicherung in Batterien...


In [52]:
train.shape

(8000, 11)

## Hyperparameters

In [53]:
# "text" or "html"
TEXT_COL = "text"

# "group_representative", "group_representative_label", "industry", "industry_label" or "group"
CLASS_COL = "group_representative"
CLASS_NAMES = "group_representative_label"

MAX_DOCUMENT_FREQUENCY = 1.
MAX_FEATURES = 10000
NGRAM_RANGE = (1,1)
LOWERCASE = False
#STOP_WORDS = get_stop_words("de")
STOP_WORDS = None

TAG_LIST = ['a', 'b', 'em', 'h1', 'h2', 'h3', 'i', 'li', 'p', 'strong', 'title']

## Add Meta-Tag information to plain text

In [43]:
train["text"] = train["text"] + train["meta"]

## Trim HTML

In [9]:
train2 = train.head(10)

In [17]:
%%time
train["html"] = train["html"].apply(lambda x: trim_html(x, tag_list = TAG_LIST, tagless_output_string=True))

ValueError: Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.

### Vectorizing text

In [54]:
%%time

train_text = train[TEXT_COL]
train_labels = train[CLASS_COL].values

vectorizer = CountVectorizer(max_df=MAX_DOCUMENT_FREQUENCY,
                             lowercase=LOWERCASE,
                             max_features=MAX_FEATURES,
                             ngram_range=NGRAM_RANGE,
                             stop_words=STOP_WORDS,
                            tokenizer=tokenizing_html)
transformer = TfidfTransformer()

vector = vectorizer.fit_transform(train_text)
train_vector = transformer.fit_transform(vector)

CPU times: user 8.46 s, sys: 63.5 ms, total: 8.52 s
Wall time: 8.51 s


# Test Dataset

In [55]:
%%time
test = pd.read_csv(TEST_PATH_CSV)
    
test_vector = vectorizer.transform(test[TEXT_COL].values)
test_vector = transformer.transform(test_vector)
test_labels = test[CLASS_COL].values

CPU times: user 2.18 s, sys: 28.6 ms, total: 2.21 s
Wall time: 2.21 s


# SGD

In [56]:
%%time
print("SGD CLF", "\n-------------------------")
# training
clf = SGDClassifier()
clf.fit(train_vector, train_labels)

# prediction
train_preds = clf.predict(test_vector)

# evaluation
precision = precision_score(test_labels, train_preds, average="macro", zero_division=0)
recall = recall_score(test_labels, train_preds, average="macro", zero_division=0)
f1 = f1_score(test_labels, train_preds, average="macro", zero_division=0)
clf1_f1 = np.round(f1, decimals=4)
clf1_precision = np.round(precision, decimals=4)

print(np.round(precision, decimals=4), "\tPrecision")
print(np.round(recall, decimals=4), "\tRecall")
print(np.round(f1, decimals=4), "\tF1")
print()

clf1_report = classification_report(test_labels, 
                                   train_preds, 
                                   target_names = np.unique(test[CLASS_NAMES]), 
                                   zero_division = 0)

SGD CLF 
-------------------------
0.6301 	Precision
0.5634 	Recall
0.5841 	F1

CPU times: user 1.09 s, sys: 0 ns, total: 1.09 s
Wall time: 1.08 s


# LSVM

In [57]:
%%time
print("LSVM CLF", "\n-------------------------")
# training
clf = LinearSVC()
clf.fit(train_vector, train_labels)

# prediction
train_preds = clf.predict(test_vector)

# evaluation
precision = precision_score(test_labels, train_preds, average="macro", zero_division=0)
recall = recall_score(test_labels, train_preds, average="macro", zero_division=0)
f1 = f1_score(test_labels, train_preds, average="macro", zero_division=0)
clf2_f1 = np.round(f1, decimals=4)
clf2_precision = np.round(precision, decimals=4)

print(np.round(precision, decimals=4), "\tPrecision")
print(np.round(recall, decimals=4), "\tRecall")
print(np.round(f1, decimals=4), "\tF1")
print()

clf2_report = classification_report(test_labels, 
                                   train_preds, 
                                   target_names = np.unique(test[CLASS_NAMES]),
                                   zero_division = 0)

LSVM CLF 
-------------------------
0.6257 	Precision
0.5523 	Recall
0.5778 	F1

CPU times: user 2.44 s, sys: 8.03 ms, total: 2.45 s
Wall time: 2.44 s


## Summary: Classification Results

In [58]:
result = "| "

if TEXT_COL == "text":
    result += "Plain Text"
else:
    result += "HTML"
    
if STOP_WORDS is None:
    result += " (kept stop words)"
    
if MAX_FEATURES is None:
    result += " (all features)"
else:
    result += f" ({MAX_FEATURES} features)"
    
if NGRAM_RANGE != (1,1):
    result += f" ({NGRAM_RANGE} ngrams)"
    
if ROWS:
    result += f" ({ROWS[1:]} rows)"
    
            
result += f" | **{clf1_f1}** ({clf1_precision}) | **{clf2_f1}** ({clf2_precision}) |"
print(CLASS_COL)
print()
print(result)

group_representative

| Plain Text (kept stop words) (10000 features) (10000 rows) | **0.5841** (0.6301) | **0.5778** (0.6257) |


# Confusion Matrix

TODO: label und text names und so; allg. änderungen von oben hier ergänzen

In [ ]:
NORMALIZE_CM = True
INDUSTRY_TRESHOLD = 250
PLT_SCALING_FACTOR = 0.8

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

filtered_train = train.groupby(CLASS_COL).filter(lambda x: len(x)>INDUSTRY_TRESHOLD)
remaining_industries = filtered_train[CLASS_NAMES].drop_duplicates().tolist()


cnf_matrix = confusion_matrix(test_labels, train_preds)

classes = train[CLASS_COL].drop_duplicates().tolist()

cnf_df = pd.DataFrame(cnf_matrix, index=classes, columns=classes)
cnf_df = cnf_df[remaining_industries]
cnf_df = cnf_df.loc[remaining_industries]

In [ ]:
plt.figure(figsize=(10*PLT_SCALING_FACTOR, 8*PLT_SCALING_FACTOR))

if NORMALIZE_CM:
    normalized_cnf_df = cnf_df.astype('float') / cnf_df.sum(axis=1)[:, np.newaxis]
    sns.heatmap(normalized_cnf_df, annot=True, cmap=sns.color_palette("Blues"), fmt='.2f')
else:
    sns.heatmap(cnf_df, annot=True, cmap=sns.color_palette("Blues"), fmt='g')
plt.tight_layout()